# Import Libraries

In [1]:
import os
import pickle
import torch
import torch.nn as nn
import pandas as pd
import importlib
from collections import defaultdict
import numpy as np

# All imports are done relative to the 
# root of the project.
#
project_root = '../../'
if 'change_directory_to_root' not in globals():
    change_directory_to_root = True
    os.chdir(project_root)

# Imports own modules.
#
import scripts.case_study.OptimizeBESS as OptimizeBESS
import scripts.Utils as utils
import scripts.Simulation_config as Simulation_config
from scripts.Simulation_config import *


# Import Price Signal

In [2]:
# Import energy price in €/Wh
#
with open('data/exaa_prices_1h.pkl', 'rb') as f:
    price_signal = pickle.load(f)


# Import last Load Forecasts

In [ ]:
# Import electrical load forecast in W
#
importlib.reload(utils)
importlib.reload(Simulation_config)

########
# Setup the test setup configuration
#
resuts_filename = 'scripts/outputs/all_train_histories.pkl'
expected_configs = [
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._1_HOUSEHOLD, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._2_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._10_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._50_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._100_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT),
    ]
########

# Get the stored results file from the last test run
result_dict = utils.Evaluate_Models.print_results(resuts_filename, print_style = 'predicted_profiles')

# Check, if all data are available
# and create a nested dictionary of shape 'profiles_by_community_size[community_size][model][community_id]'
#
profiles_by_community_size = defaultdict(dict)
for expected_config in expected_configs:
    for available_config in result_dict:
        if expected_config == available_config:
            community_size = expected_config.aggregation_Count[0]
            profiles_by_community_size[community_size] = result_dict[expected_config]
assert len(profiles_by_community_size) == len(expected_configs), \
    f"Not all expected test-runs found: {len(profiles_by_community_size)} != {len(expected_configs)}."


## MILP Optimization

In [4]:
importlib.reload(OptimizeBESS)

# Run the MILP optimziation for all communities and model sizes.
#
costs_per_community_size = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
for community_size, profiles_per_model in profiles_by_community_size.items():
    for model_name, profiles_per_community in profiles_per_model.items():
        print(f'Optimize community size {community_size} with model "{model_name}"')
        for community_id, profile in enumerate(profiles_per_community):
            perfect_prediction = profiles_per_model['Perfect'][community_id]
            myOptimization = OptimizeBESS.OptimizeBESS( profile, 
                                                        community_size, 
                                                        price_signal, 
                                                        perfect_prediction)
            optimization_result = myOptimization.run()
            costs_per_community_size[community_size][model_name][community_id] = optimization_result


Run community size 1 with model "Perfect"


Run community size 1 with model "KNN"
Run community size 1 with model "PersistencePrediction"
Run community size 1 with model "xLSTM"
Run community size 1 with model "LSTM"
Run community size 1 with model "Transformer_Encoder_Only"
Run community size 2 with model "Perfect"
Run community size 2 with model "KNN"
Run community size 2 with model "PersistencePrediction"
Run community size 2 with model "xLSTM"
Run community size 2 with model "LSTM"
Run community size 2 with model "Transformer_Encoder_Only"
Run community size 10 with model "Perfect"
Run community size 10 with model "KNN"
Run community size 10 with model "PersistencePrediction"
Run community size 10 with model "xLSTM"
Run community size 10 with model "LSTM"
Run community size 10 with model "Transformer_Encoder_Only"
Run community size 50 with model "Perfect"
Run community size 50 with model "KNN"
Run community size 50 with model "PersistencePrediction"
Run community size 50 with model "xLSTM"
Run community size 50 with model "

# Evaluation

In [5]:
# Calc the normalized mean absolut error:
#
def calc_nMAE(Y_perfect, Y_model):
    loss_fn = nn.L1Loss()
    Y_perfect = torch.Tensor(Y_perfect)
    Y_model = torch.Tensor(Y_model)
    loss = loss_fn(Y_perfect, Y_model)
    reference = float(torch.mean(Y_perfect))
    nMAE = 100.0 * loss / reference
    return float(nMAE)

# Print all Results
#
for nr_of_households, costs_per_model in costs_per_community_size.items():
    print('\nCommunity Size:', nr_of_households, 'household(s).')
    print(42*'-')
    
    # Calculate the average costs without optimization
    reference_costs = []
    perfect_profiles = profiles_by_community_size[nr_of_households]['Perfect']
    for perfect_profile in perfect_profiles:
        assert price_signal.shape == perfect_profile.shape, "Arrays must be the same shape"
        assert price_signal.ndim == 1, "price_signal must be a 1D array"
        costs = np.sum(price_signal * perfect_profile)
        reference_costs.append(costs)
    avg_reference_costs = np.mean(reference_costs)
    
    # Store the result
    results = [{
            "Model": 'Unoptimized',
            "nMAE (%)": '',
            "Costs (€)": round(avg_reference_costs, 2),
            "Savings (€)": '',
            "Savings (%)": ''
        }]
    
    for model_name, costs_per_community in costs_per_model.items():

        # Calc the average costs with optimization per model over all communities
        all_costs = list(costs_per_community.values())
        costs_avg = np.mean(all_costs)
        savings_avg = np.mean([(avg_reference_costs - costs) for costs in all_costs])
        savings_percent_avg = np.mean([100*(avg_reference_costs - costs)/avg_reference_costs for costs in all_costs])
        profile_per_community = profiles_by_community_size[nr_of_households][model_name]
        nMAE_avg = np.mean([calc_nMAE(perfect_profiles[i], profile)
                            for i, profile in enumerate(profile_per_community)])

        # Optionally rename model for printing
        if model_name == 'Transformer_Encoder_Only':
            model_name = 'Transformer'
        elif model_name == 'PersistencePrediction':
            model_name = 'Persistence'

        results.append({
            "Model": model_name,
            "nMAE (%)": round(nMAE_avg, 2),
            "Costs (€)": round(costs_avg, 2),
            "Savings (€)": round(savings_avg, 2),
            "Savings (%)": round(savings_percent_avg, 2)
        })

    df = pd.DataFrame(results)
    print(df.to_string(index=False))



Community Size: 1 household(s).
------------------------------------------
      Model nMAE (%)  Costs (€) Savings (€) Savings (%)
Unoptimized              299.06                        
    Perfect      0.0     280.65       18.41        6.16
        KNN    53.84     295.38        3.68        1.23
Persistence    57.25     296.85        2.21        0.74
      xLSTM    50.32     293.22        5.84        1.95
       LSTM    50.06     292.70        6.36        2.13
Transformer     50.7     292.67        6.38        2.13

Community Size: 2 household(s).
------------------------------------------
      Model nMAE (%)  Costs (€) Savings (€) Savings (%)
Unoptimized              495.71                        
    Perfect      0.0     456.94       38.78        7.82
        KNN    39.04     480.94       14.77        2.98
Persistence    44.98     483.49       12.22        2.47
      xLSTM    37.57     477.25       18.47        3.72
       LSTM    37.46     478.62       17.09        3.45
Transfor